In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *
from _utils.normalized_price import calculate_normalised_price_dict
from _utils.strategies.trend_asset_classes import (
    calculate_asset_class_price_dict,
    calculate_relative_price_dict
    )

from _utils.strategies.carry_asset_classes import (
    relative_carry,
    )


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [2]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data


def get_fx_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    price = dict(
        [
            (instrument_code, data_for_instrument.price)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )


    return price

In [3]:
# função auxiliar
def align_dict_to_index(data_dict, target_index):
    for inst in data_dict.keys():
        df = data_dict[inst]

        data_dict[inst] = df.reindex(target_index).ffill().bfill()

    return data_dict




In [4]:
adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict(["sp500", "eurostx", "us10", "us2"])

ref = "eurostx"

target_index = adjusted_prices_dict[ref].index
adjusted_prices_dict  = align_dict_to_index(adjusted_prices_dict,  target_index)
current_prices_dict   = align_dict_to_index(current_prices_dict,   target_index)
carry_prices_dict     = align_dict_to_index(carry_prices_dict,     target_index)
fx_data_dict = get_fx_data_dict(['eur_fx'])
multipliers = dict(sp500=5, eurostx=10, us10=1000, us2=2000)
risk_target_tau = 0.2
asset_class_groupings = dict(bonds=["us2", "us10"], stocks=["sp500", "eurostx"])

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, eurostx=fx_data_dict['eur_fx'], us10=1, us2=1))

capital = 2000000
idm = 2
instrument_weights = dict(sp500=0.25, eurostx=0.25, us10=0.25, us2=0.25)
cost_per_contract_dict = dict(sp500=0.875, eurostx=6.8, us10=9.5, us2=5.5)


std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices_dict,
current_prices=current_prices_dict,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [5]:
average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

normalised_price_dict = calculate_normalised_price_dict(
       adjusted_prices_dict=adjusted_prices_dict,
       std_dev_dict=std_dev_dict,
   )


asset_class_price_dict = calculate_asset_class_price_dict(
       normalised_price_dict=normalised_price_dict,
       asset_class_groupings=asset_class_groupings,
   )

relative_price_dict = calculate_relative_price_dict(
        adjusted_prices_dict=adjusted_prices_dict,
        std_dev_dict=std_dev_dict,
        asset_class_groupings=asset_class_groupings,
    )

rule_trend_carry_spec = [
        dict(function="carry", span=5),
        dict(function="carry", span=20),
        dict(function="carry", span=60),
        dict(function="carry", span=120),
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]


rules_spec_ewmac = [
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]


rules_spec_carry = [
    dict(function="carry", span=5),
    dict(function="carry", span=20),
    dict(function="carry", span=60),
    dict(function="carry", span=120),
]


# ==============================================

list_of_rules = [
            dict(function=relative_carry, span=90, scalar=50, asset_class_groupings=asset_class_groupings),
        ]

position_contracts_relative_carry_class_dict = calculate_position_dict_with_forecast_from_function_applied(
    adjusted_prices_dict=adjusted_prices_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    list_of_rules=list_of_rules,
)
    

buffered_position_relative_carry_class_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_relative_carry_class_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_relative_carry_class_dict = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_relative_carry_class_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)


port_trend_relative_carry_class_return = aggregate_returns(
    perc_return_relative_carry_class_dict)
port_trend_relative_carry_class_return.sort_index(inplace=True)

# ============================

position_contracts_relative_return_class_dict = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=relative_price_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_ewmac,
)

buffered_position_relative_return_class_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_relative_return_class_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_relative_return_class_dict = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_relative_return_class_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)


port_trend_relative_return_class_return = aggregate_returns(
    perc_return_relative_return_class_dict)

port_trend_relative_return_class_return.sort_index(inplace=True)

# ==============

position_contracts_asset_class_dict = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=asset_class_price_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_ewmac,
)

buffered_position_asset_class_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_asset_class_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_asset_class_dict = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_asset_class_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)


port_trend_asset_class_return = aggregate_returns(
    perc_return_asset_class_dict)

port_trend_asset_class_return.sort_index(inplace=True)


#===========
position_contracts_normalized_trend_dict = calculate_position_dict_with_forecast_applied(
        adjusted_prices_dict=normalised_price_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        rule_spec=rules_spec_ewmac,
    )


buffered_position_normalized_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_normalized_trend_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_normalized_trend_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_normalized_trend_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)




port_normalized_trend_return = aggregate_returns(
    perc_return_normalized_trend_dict)

port_normalized_trend_return.sort_index(inplace=True)

# ===============

position_contracts_carry_trend_dict = calculate_position_dict_with_forecast_applied(
        adjusted_prices_dict=
        adjusted_prices_dict,
        carry_prices_dict=
        carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=
        average_position_contracts_dict,
        rule_spec=rule_trend_carry_spec,
    )


buffered_position_carry_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_carry_trend_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_carry_trend_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_carry_trend_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)

port_carry_trend_return = aggregate_returns(
    perc_return_carry_trend_dict)

port_carry_trend_return.sort_index(inplace=True)

#=================================


position_contracts_dict_ewmac = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=adjusted_prices_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_ewmac,
)

buffered_position_dict_ewmac = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_dict_ewmac,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_dict_ewmac = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_dict_ewmac,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)

perc_return_aggregated_ewmac = aggregate_returns(perc_return_dict_ewmac)

position_contracts_dict_carry = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=adjusted_prices_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_carry,
)

buffered_position_dict_carry = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_dict_carry,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_dict_carry = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_dict_carry,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)

perc_return_aggregated_carry = aggregate_returns(perc_return_dict_carry)

starting_portfolio = (
    perc_return_aggregated_ewmac * 0.6 + perc_return_aggregated_carry * 0.4
)

relative_performance = perc_return_aggregated_ewmac - perc_return_aggregated_carry
rolling_12_month = relative_performance.rolling(BUSINESS_DAYS_IN_YEAR).mean()

relative_performance = perc_return_aggregated_ewmac - perc_return_aggregated_carry
rolling_12_month = (
    relative_performance.rolling(BUSINESS_DAYS_IN_YEAR).sum() / risk_target_tau
)

    # W t = EWMA span=30 (min(1, max(0, 0.5 + RP t ÷ 2)))
raw_weighting = 0.5 + rolling_12_month / 2
clipped_weighting = raw_weighting.clip(lower=0, upper=1)
smoothed_weighting = clipped_weighting.ewm(30).mean()

weighted_portfolio = (
    perc_return_aggregated_ewmac * 0.5 * smoothed_weighting
    + perc_return_aggregated_carry * 0.5 * (1 - smoothed_weighting)
)




In [6]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=port_trend_relative_carry_class_return.index,
        y=port_trend_relative_carry_class_return.cumsum()*100,
        name="Relative Carry with Asset Classes",
        line=dict(color="purple"),
    )
)


fig.add_trace(
    go.Scatter(
        x=port_trend_relative_return_class_return.index,
        y=port_trend_relative_return_class_return.cumsum()*100,
        name="Relative Trend with Asset Classes",
        line=dict(color="orange"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_trend_asset_class_return.index,
        y=port_trend_asset_class_return.cumsum()*100,
        name="Normalized Trend with Asset Classes",
        line=dict(color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_normalized_trend_return.index,
        y=port_normalized_trend_return.cumsum()*100,
        name="Normalized Trend",
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_carry_trend_return.index,
        y=port_carry_trend_return.cumsum()*100,
        name="Carry & Trend",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=weighted_portfolio.index,
        y=weighted_portfolio.cumsum()*100,
        name="Carry & Trend with relative performance",
        line=dict(color="blue"),
    )
)



fig.update_layout(
    title="Strategies for Future Contracts Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0.0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Relative Carry with Asset Class returns Frequência Anual")
print(calculate_stats(port_trend_relative_carry_class_return, freq=YEAR))
print("Estatísticas Portfolio Relative Trend with Asset Class returns Frequência Anual")
print(calculate_stats(port_trend_relative_return_class_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Normalized Trend with Asset Class returns Frequência Anual")
print(calculate_stats(port_trend_asset_class_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Normalized Trend Frequência Anual")
print(calculate_stats(port_normalized_trend_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Carry and Trend with Relative Alocation Frequência Anual")
print(calculate_stats(weighted_portfolio, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Carry and Trend Frequência Anual")
print(calculate_stats(port_carry_trend_return, freq=YEAR))



Estatísticas de Desempenho
Estatísticas Portfolio Relative Carry with Asset Class returns Frequência Anual
{'ann_mean': -25.096013706885106, 'ann_std': 2.195743105145193, 'sharpe': -11.42939428937687, 'skew': -0.19669548125558942, 'avg_drawdown': 0.3065051928940348, 'max_drawdown': 0.8767709843562793, 'quant_lower': 0.6881280321304433, 'quant_upper': 0.6058545614869505}
Estatísticas Portfolio Relative Trend with Asset Class returns Frequência Anual
{'ann_mean': 12.091663593543606, 'ann_std': 2.9012122418978397, 'sharpe': 4.167796970839264, 'skew': -1.0688264031505246, 'avg_drawdown': 0.020597981943544084, 'max_drawdown': 0.14695684615857638, 'quant_lower': 17.745360662154894, 'quant_upper': 0.6619493196711734}
Estatísticas Portfolio Normalized Trend with Asset Class returns Frequência Anual
{'ann_mean': 79.49617860356962, 'ann_std': 6.460453225772737, 'sharpe': 12.305046693385982, 'skew': 0.4757780993382248, 'avg_drawdown': 0.03962267570694294, 'max_drawdown': 0.26893202825015816, 'qua